In [1]:
import pandas as pd
import math

In [2]:
answers = pd.read_csv("answers.csv", escapechar='\\',index_col=0) 

In [3]:
f1_scores = pd.read_csv("f1_scores.csv", escapechar='\\',index_col=0) 

In [4]:
answers_f1 = pd.merge(answers, f1_scores, on='qid')

In [10]:
null_odds = pd.read_json("squad_null_odds.json",typ='series').to_frame(name="null_odds")
null_odds.index.name = 'qid'

In [5]:
null_odds_confirm = pd.read_json("squad_null_odds_confirm.json",typ='series').to_frame(name="null_odds_confirm")
null_odds_confirm.index.name = 'qid'

In [8]:
answers_f1_null_odds_confirm = pd.merge(answers_f1, null_odds_confirm, on='qid')

In [11]:
answers_f1_null_odds_confirm_all = pd.merge(answers_f1_null_odds_confirm, null_odds, on='qid')

In [8]:
answers_f1_null_odds.groupby('score')['score'].value_counts()

score  score
0      0         328
1      1        1002
Name: score, dtype: int64

In [9]:
answers_f1_null_odds[answers_f1_null_odds["question"].str.contains("What was|What is", na=False)] \
    .groupby('score')['score'].value_counts()

score  score
0      0         52
1      1        131
Name: score, dtype: int64

In [10]:
52/131

0.3969465648854962

In [11]:
328/1002

0.3273453093812375

In [12]:
answers_f1_null_odds[(answers_f1_null_odds["question"] \
                      .str.contains("What was|What is", na=False)) \
                     & (answers_f1_null_odds['null_odds'] > -5.0) \
                     & (answers_f1_null_odds['null_odds'] < -2.0)] \
    .groupby('score')['score'].value_counts()



score  score
0      0        25
1      1        23
Name: score, dtype: int64

In [15]:
# About half of the answers that it got wrong are > -2.0 and < 5.0

In [16]:
scores_0_5 = answers_f1_null_odds[(answers_f1_null_odds['null_odds'] > -5.0) \
                     & (answers_f1_null_odds['null_odds'] < -0.0)] \
    .groupby('score')['score'].value_counts()
incorrect_count = scores_0_5.loc[0].at[0]
correct_count = scores_0_5.loc[1].at[1]
incorrect_count

59

In [57]:
# Correct but weak confidence
scores = answers_f1_null_odds[(answers_f1_null_odds['null_odds'] > -2.75) \
                     & (answers_f1_null_odds['null_odds'] < 0.0)]
scores.loc[((scores['score'] == 0) & (scores['scorey'] > 0 ))]


,question,gold,pred,scorex,scorey,score,null_odds
qid,,,,,,,
5705e26d75f01819005e76d8,The reasons for the las two counties to be added are based on what?,historical political divisions,historical political divisions,1.0,1.000000,0,-0.710475
5705e99452bb891400689689,What country are all the counties in?,United States,United States,1.0,1.000000,0,-1.343167
570607f575f01819005e78b5,What is the population of the second largest city in California?,1.3 million,1.3 million,1.0,1.000000,0,-1.487548
57060a1175f01819005e78d6,What industry has managed to survive major military spending cutbacks?,aerospace,aerospace,1.0,1.000000,0,-2.532187
57060eaf75f01819005e7913,What other business district does Orange County envelop outside of Downtown Santa Ana and Newport Center?,South Coast Metro,South Coast Metro,1.0,1.000000,0,-2.263996
5706111a52bb89140068984f,"A single line connects San Bernardino, Riverside and what other county?",Orange,Orange,1.0,1.000000,0,-2.012276
57106185b654c5140001f8db,The term Huguenot was originally meant to confer?,derision,derision,1.0,1.000000,0,-2.441573
57107d73b654c5140001f91f,Which French kind issued this declaration?,Louis XIV,Louis XIV,1.0,1.000000,0,-2.188953
57107e6ca58dae1900cd69f3,What event was the worst example of Huguenot persecution?,St. Bartholomew's Day massacre,St. Bartholomew's Day massacre,1.0,1.000000,0,-0.093341


In [16]:
def isNotNaN(num):
    return num == num

In [63]:
# Incorrect but weak confidence
scores = answers_f1_null_odds[(answers_f1_null_odds['null_odds'] > -2.75) \
                     & (answers_f1_null_odds['null_odds'] < 0.0)]
scores.loc[((scores['score'] == 0) & (scores['scorey'] == 0 ))]


,question,gold,pred,scorex,scorey,score,null_odds
qid,,,,,,,
57060a1175f01819005e78d3,What characteristic best describes the agricultural regions that could be found?,richest,cattle and citrus,0.0,0.0,0,-2.189356
57108198b654c5140001f93b,What country currently has a group who call themselves Huguenots?,Australia,France,0.0,0.0,0,-1.923978
5728d4c03acd2414000dffa3,What civil rights movement in the US was known for it's disobedience?,American Civil Rights Movement,the Singing Revolution,0.0,0.0,0,-2.316794
5728d9403acd2414000e001d,What type of person can not be attributed civil disobedience?,head of government,private citizen,0.0,0.0,0,-0.188889
572812e74b864d19001643cf,What was Thoreau's punishment for not paying his taxes?,imprisonment,Resign,0.0,0.0,0,-2.029530
5728151b4b864d190016442a,Brownlee justifies civil disobedience toward what branch of the government?,legal system,governmental entities,0.0,0.0,0,-0.434944
5728e5f43acd2414000e0172,What is one way to use pure speech to reach as much people as possible to protest?,broadcasting,criminalized behavior,0.0,0.0,0,-1.422025
5728e715ff5b5019007da914,What is a type of disobedience against the federal government?,refusals to pay taxes,sit-ins,0.0,0.0,0,-1.275316
5728eef92ca10214002daab1,What is usually the goal of taking a plea bargain?,no jail time,solidarity,0.0,0.0,0,-1.839375


In [13]:
def isNaN(num):
    return num != num

In [61]:
pd.set_option("display.max_colwidth", 200)

In [15]:
# Too confident - fixes 3 of 11
scores = answers_f1_null_odds_confirm_all[(answers_f1_null_odds_confirm_all['null_odds'] > -5.00) \
                     & (answers_f1_null_odds_confirm_all['null_odds'] <= -2.75)]
scores.loc[((scores['score'] == 0) & isNaN(scores['gold']))]


,question,gold,pred,scorex,scorey,score,null_odds_confirm,null_odds
qid,,,,,,,,
5ad0178577cf76001a68698b,What additional two counties make ten counties...,NaN,Kern and San Luis Obispo,0.0,0.0,0,-2.993419,-4.916696
5ad01c8877cf76001a686a65,What desert is to the south near Nevada?,NaN,Mojave Desert,0.0,0.0,0,2.869363,-3.856768
5ad01e4677cf76001a686aa6,What are two of the three major cities located...,NaN,Los Angeles and San Diego,0.0,0.0,0,-4.519358,-3.066281
5ad01f0f77cf76001a686ac3,How many populous counties are in the United S...,NaN,15,0.0,0.0,0,-3.561306,-3.153225
5ad0206677cf76001a686afb,What three industries are centered in Hollywood?,NaN,"motion picture, television, and music",0.0,0.0,0,0.214466,-2.894037
5ad0206677cf76001a686afd,What major companies are headquartered in Los ...,NaN,"Walt Disney Company (which also owns ABC), Son...",0.0,0.0,0,-4.258379,-3.582415
5ad0220a77cf76001a686b2a,Where are No Fear and RCVA headquartered?,NaN,Southern California,0.0,0.0,0,-1.799177,-3.894468
5ad0394877cf76001a686e59,What area has the most diversity of anywhere i...,NaN,Southern California,0.0,0.0,0,-5.879231,-3.513397
5ad0410977cf76001a686efe,What are regions anchored by that are recogniz...,NaN,a city,0.0,0.0,0,-5.276679,-3.841857


In [17]:
# Not confident enough - fixes 4 out of 5
scores = answers_f1_null_odds_confirm_all[(answers_f1_null_odds_confirm_all['null_odds'] > -2.75) \
                     & (answers_f1_null_odds_confirm_all['null_odds'] <= 0.0)]
scores.loc[((scores['score'] == 0) & isNotNaN(scores['gold']))]

,question,gold,pred,scorex,scorey,score,null_odds_confirm,null_odds
qid,,,,,,,,
5705e26d75f01819005e76d8,The reasons for the las two counties to be add...,historical political divisions,historical political divisions,1.0,1.0,0,-1.866194,-0.710475
5705e99452bb891400689689,What country are all the counties in?,United States,United States,1.0,1.0,0,-3.625940,-1.343167
570607f575f01819005e78b5,What is the population of the second largest c...,1.3 million,1.3 million,1.0,1.0,0,-3.251086,-1.487548
57060a1175f01819005e78d3,What characteristic best describes the agricul...,richest,cattle and citrus,0.0,0.0,0,-3.581009,-2.189356
57060a1175f01819005e78d6,What industry has managed to survive major mil...,aerospace,aerospace,1.0,1.0,0,-6.371802,-2.532187


In [18]:
# Breaks good scores? No answer cases - breaks 7 of 10!!!
scores = answers_f1_null_odds_confirm_all[(answers_f1_null_odds_confirm_all['null_odds'] > -2.75) \
                     & (answers_f1_null_odds_confirm_all['null_odds'] <= 0.0)]
scores.loc[((scores['score'] == 1) & isNaN(scores['gold']))]

,question,gold,pred,scorex,scorey,score,null_odds_confirm,null_odds
qid,,,,,,,,
5ad0178577cf76001a68698c,Where is the center of economic activity for t...,NaN,Southern California,0.0,0.0,1,-5.609703,-2.491946
5ad01a6277cf76001a686a0e,How many people live in Riverside?,NaN,over four million,0.0,0.0,1,-3.439244,-1.934785
5ad01f0f77cf76001a686ac4,"What county are Los Angeles, Orange, San Diego...",NaN,the five most populous in the state,0.0,0.0,1,-5.343992,-2.386950
5ad0228a77cf76001a686b58,Who visits Palm Springs for the beaches?,NaN,locals and tourists,0.0,0.0,1,-4.169446,-1.702614
5ad02cde77cf76001a686cc6,What country did California once rule?,NaN,Mexico,0.0,0.0,1,3.764789,-1.201254
5ad02ff177cf76001a686da2,What type of club is the California Automobile...,NaN,AAA Auto Clubs,0.0,0.0,1,-4.365382,-1.592745
5ad02ff177cf76001a686da5,What type of groups divides California into on...,NaN,regional tourism groups,0.0,0.0,1,-4.484828,-2.215286
5ad0316a77cf76001a686de4,What are the second and third most populated m...,NaN,Great Lakes Megalopolis,0.0,0.0,1,4.701613,-2.282655
5ad0394877cf76001a686e57,What ocean has the Transverse and Peninsular R...,NaN,Pacific,0.0,0.0,1,0.044508,-1.606575


In [20]:
# Breaks good scores? Answer cases - breaks 6 cases
scores = answers_f1_null_odds_confirm_all[(answers_f1_null_odds_confirm_all['null_odds'] > -5.00) \
                     & (answers_f1_null_odds_confirm_all['null_odds'] <= -2.75)]
scores.loc[((scores['score'] == 1) & isNotNaN(scores['gold']))]

,question,gold,pred,scorex,scorey,score,null_odds_confirm,null_odds
qid,,,,,,,,
5705e26d75f01819005e76d5,"Despite being traditionall described as ""eight...",10 counties,10,1.0,1.0,1,-2.162720,-3.119258
5705e33f52bb89140068964e,What is the name of the state that the megareg...,Nevada,Nevada,1.0,1.0,1,-3.798012,-3.798012
5705e4fe75f01819005e7704,What is the name of the water body that is fou...,Colorado River,Colorado River,1.0,1.0,1,-0.860442,-4.414096
5705e4fe75f01819005e7705,What is the name of the desert on the border o...,Colorado Desert,Colorado Desert,1.0,1.0,1,-3.072966,-4.744115
5705e63175f01819005e7723,What is the eighth most populous city in the n...,San Diego,San Diego,1.0,1.0,1,-4.160767,-4.540154
5705eb3375f01819005e7768,"Other than Universal and Warner Brothers, what...",Sony,Sony,1.0,1.0,1,2.606699,-4.044423
5705ec1675f01819005e776e,"Other than surf, what other culture is souther...",skateboard,skateboard,1.0,1.0,1,-3.081206,-4.823408
5705ec1675f01819005e7772,What is the shortened name of the annual yacht...,Transpac,Transpac,1.0,1.0,1,-5.961582,-4.246503
5705edcd52bb8914006896ce,Point Conception is an example of a landmark a...,northern,Tehachapi Mountains,1.0,1.0,1,-3.811043,-4.794703


In [312]:
scores = answers_f1_null_odds[(answers_f1_null_odds['null_odds'] M -5.00) \
                     & (answers_f1_null_odds['null_odds'] <= -2.75)]
scores.loc[((scores['score'] == 0))].to_csv('non_null_range.csv')

SyntaxError: invalid syntax (<ipython-input-312-584469f9afc6>, line 1)

In [270]:
 answers_f1_null_odds.count()

question     1330
gold          700
pred         1330
scorex       1330
scorey       1329
score        1330
null_odds    1330
dtype: int64

In [314]:
scores = answers_f1_null_odds[(answers_f1_null_odds['null_odds'] < 0) \
                     & (answers_f1_null_odds['null_odds'] > -2.75)]
scores.loc[((scores['score'] == 1))].to_csv('null_range_correct.csv')

In [17]:
# Export all 'unsure' answers
scores_0_5 = answers_f1_null_odds[(answers_f1_null_odds['null_odds'] > -5.0) \
                     & (answers_f1_null_odds['null_odds'] < -0.0)]
scores_0_5.to_csv('scores_unsure.csv')

In [49]:
from csv import reader

with open("confirm_questions.csv", "r") as read_obj:
        csv_reader = reader(read_obj)
        confirm_questions = dict(csv_reader)
        
#confirm_questions = pd.read_csv("confirm_questions.csv", escapechar='\\',index_col=0) 
type(confirm_questions)
confirm_questions["5705e99452bb891400689689"]

'Are all the counties in United States?'

In [65]:
import json
with open('dev.json', 'r') as f:
    data = json.load(f)
type(data)
type(data["data"][0])


for d in data["data"]:
    for p in d["paragraphs"]:
        new_questions = list()
        for q in p["qas"]:
            if q["id"] in confirm_questions:
                # add this to dictionary
                q["question"] = confirm_questions[q["id"]]
                new_questions.append(q)
        p["qas"] = new_questions

            
with open('dev_confirm.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [7]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz --no-deps

     |████████████████████████████████| 827.9MB 52kB/s  eta 0:00:015   |███▌                            | 91.3MB 834kB/s eta 0:14:43     |████▊                           | 122.9MB 536kB/s eta 0:21:55     |█████▋                          | 145.0MB 5.2MB/s eta 0:02:13     |███████████▍                    | 294.2MB 3.6MB/s eta 0:02:28     |███████████▊                    | 302.7MB 4.1MB/s eta 0:02:08     |█████████████▏                  | 340.4MB 8.3MB/s eta 0:00:59     |█████████████▊                  | 353.6MB 4.0MB/s eta 0:01:58     |█████████████████▎              | 445.6MB 1.5MB/s eta 0:04:11██████████▋             | 482.6MB 14.2MB/s eta 0:00:25     |███████████████████▌            | 503.8MB 3.9MB/s eta 0:01:25  |████████████████████▋           | 532.6MB 494kB/s eta 0:09:57  |█████████████████████           | 540.6MB 548kB/s eta 0:08:44     |█████████████████████▏          | 547.5MB 931kB/s eta 0:05:02     |█████████████████████▎          | 551.3MB 1.2MB/s eta 0:03:52     |██████████

In [1]:
import spacy

In [9]:
nlp = spacy.load('en_core_web_lg')

What PRON WDT det Xxxx True True
country NOUN NN nsubj xxxx True False
are AUX VBP ROOT xxx True True
all DET PDT predet xxx True True
the DET DT det xxx True True
counties NOUN NNS attr xxxx True False
in ADP IN prep xx True True
? PUNCT . punct ? False False


In [23]:
from spacy.matcher import Matcher

In [63]:

#doc = nlp("What event was the worst example of Huguenot persecution?")
#doc = nlp("What is the population of the second largest city in California?")
doc = nlp("What country are all the counties in?")
#

In [64]:
for token in doc:
    print(token.text,  token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

What PRON WDT det Xxxx True True
country NOUN NN nsubj xxxx True False
are AUX VBP ROOT xxx True True
all DET PDT predet xxx True True
the DET DT det xxx True True
counties NOUN NNS attr xxxx True False
in ADP IN prep xx True True
? PUNCT . punct ? False False


In [65]:
pattern1 = [{'POS': 'PRON'}, {'POS': 'NOUN'}]
matcher = Matcher(nlp.vocab)
matcher.add("HelloWorld", None, pattern1)
pattern2 = [{'DEP': 'ROOT'}]
matcher.add("HelloWorld", None, pattern2)

In [66]:
matches = matcher(doc)

In [67]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

15578876784678163569 HelloWorld 0 2 What country
15578876784678163569 HelloWorld 2 3 are
